In [1]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("./data/Result.csv")
df.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,TRANS_TIME,WEEK_NO,DATE,MEMBER
0,2375,26984851472,1,1004906,1,1.39,364,1631,1,2019-07-30,0
1,1929,27021022215,4,1004906,1,1.39,441,1755,1,2019-08-02,1
2,568,27021090189,4,1004906,1,1.39,446,35,1,2019-08-02,0
3,887,27031467254,5,1004906,1,1.39,298,1551,1,2019-08-03,0
4,175,27057508267,7,1004906,1,1.99,446,2303,2,2019-08-05,0


In [3]:
product = pd.read_csv("./data/product_result.csv")
product.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [4]:
df_merge_product = df.merge(product[['PRODUCT_ID','COMMODITY_DESC']], on='PRODUCT_ID')
df_merge_product.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,TRANS_TIME,WEEK_NO,DATE,MEMBER,COMMODITY_DESC
0,2375,26984851472,1,1004906,1,1.39,364,1631,1,2019-07-30,0,POTATOES
1,1929,27021022215,4,1004906,1,1.39,441,1755,1,2019-08-02,1,POTATOES
2,568,27021090189,4,1004906,1,1.39,446,35,1,2019-08-02,0,POTATOES
3,887,27031467254,5,1004906,1,1.39,298,1551,1,2019-08-03,0,POTATOES
4,175,27057508267,7,1004906,1,1.99,446,2303,2,2019-08-05,0,POTATOES


In [5]:
df_1 = df_merge_product[['household_key','BASKET_ID','COMMODITY_DESC','QUANTITY']]
df_1

,household_key,BASKET_ID,COMMODITY_DESC,QUANTITY
0,2375,26984851472,POTATOES,1
1,1929,27021022215,POTATOES,1
2,568,27021090189,POTATOES,1
3,887,27031467254,POTATOES,1
4,175,27057508267,POTATOES,1
...,...,...,...,...
2551702,1598,42305362535,MARGARINES,1
2551703,1598,42305362535,BEERS/ALES,1
2551704,1598,42305362535,BEERS/ALES,1
2551705,1598,42305362535,BEERS/ALES,1


In [6]:
household_count = df_1.groupby('household_key').BASKET_ID.nunique()
household_count = pd.DataFrame(household_count)
household_count.head()

,BASKET_ID
household_key,
1,83
2,45
3,46
4,30
5,40


In [7]:
household_product_count  = df_1.groupby(['COMMODITY_DESC','household_key']).QUANTITY.sum().reset_index().sort_values('COMMODITY_DESC',ascending=False) # Group together
household_product_count.head()

,COMMODITY_DESC,household_key,QUANTITY
284171,YOGURT,2500,118
282991,YOGURT,843,9
282980,YOGURT,825,303
282981,YOGURT,828,7
282982,YOGURT,829,43


In [8]:
ratings = household_product_count.merge(household_count,on='household_key')
ratings['rating'] = (ratings['QUANTITY']/ratings['BASKET_ID'])

bins = [0,0.005,0.008,0.01,0.02,999]
def rating(value, bins):
    for i in range(len(bins) - 1):
        if value >= bins[i] and value <= bins[i+1]:
            return i+1
    return 1

ratings['rating_label'] = ratings.rating.apply(lambda i: rating(i, bins))

In [9]:
# # 產品銷售數量
# top = ratings.groupby(['COMMODITY_DESC'])['QUANTITY']\
#         .count().reset_index().set_index('COMMODITY_DESC')\
#         .sort_values('QUANTITY',ascending=False)

# ratings = ratings[(ratings.COMMODITY_DESC.isin(top.index[0:10000]))]

In [10]:
ratings

,COMMODITY_DESC,household_key,QUANTITY,BASKET_ID,rating,rating_label
0,YOGURT,2500,118,100,1.18,5
1,WATER - CARBONATED/FLVRD DRINK,2500,8,100,0.08,5
2,WAREHOUSE SNACKS,2500,31,100,0.31,5
3,VITAMINS,2500,6,100,0.06,5
4,VEGETABLES SALAD,2500,1,100,0.01,3
...,...,...,...,...,...,...
284167,COOKIES/CONES,1897,1,1,1.00,5
284168,CITRUS,1897,1,1,1.00,5
284169,BEERS/ALES,1897,1,1,1.00,5
284170,BAKING NEEDS,1897,1,1,1.00,5


In [11]:
ratings.to_csv("./data/ratings_COMMODITY_DESC.csv",index=False)

In [12]:
DfMatrix = pd.pivot_table(ratings, values='rating_label', index='household_key', columns='COMMODITY_DESC')
# ratings_2 = DfMatrix.dropna(thresh=10,axis=1).fillna(0)# 產品評價次數不滿10次的移除
ratings_2 = DfMatrix.reset_index().set_index('household_key').rename_axis(None, axis=1)
ratings_2 = ratings_2.fillna(0) 
ratings_2

,ADULT INCONTINENCE,AIR CARE,ANALGESICS,ANTACIDS,APPAREL,APPLES,AUDIO/VIDEO PRODUCTS,AUTOMOTIVE PRODUCTS,BABY FOODS,BABY HBC,...,VEAL,VEGETABLES - ALL OTHERS,VEGETABLES - SHELF STABLE,VEGETABLES SALAD,VITAMINS,WAREHOUSE SNACKS,WATCHES/CALCULATORS/LOBBY,WATER,WATER - CARBONATED/FLVRD DRINK,YOGURT
household_key,,,,,,,,,,,,,,,,,,,,,
1,0.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0
2,5.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,...,0.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0
3,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,...,0.0,0.0,5.0,5.0,0.0,5.0,0.0,0.0,5.0,5.0
4,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
5,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0,...,0.0,5.0,5.0,5.0,0.0,5.0,0.0,0.0,5.0,5.0
2497,0.0,5.0,5.0,1.0,0.0,5.0,0.0,4.0,5.0,1.0,...,0.0,5.0,5.0,5.0,0.0,1.0,0.0,0.0,5.0,5.0
2498,0.0,5.0,5.0,5.0,2.0,4.0,0.0,0.0,0.0,0.0,...,0.0,5.0,5.0,0.0,0.0,2.0,0.0,0.0,5.0,5.0


In [13]:
# 標準化數據(對於客戶沒買過的商品不一定不喜歡)

In [14]:
def standardize(row):
    new_row = (row - row.mean()) / (row.max() - row.min())
    return new_row

ratings_std = ratings_2.apply(standardize)
# 計算餘弦相似度
item_similarity = cosine_similarity(ratings_std.T)
print(item_similarity)

[[1.         0.09454374 0.06713398 ... 0.0530283  0.03449277 0.04824983]
 [0.09454374 1.         0.28139098 ... 0.1274861  0.21825991 0.18299363]
 [0.06713398 0.28139098 1.         ... 0.08400815 0.23456407 0.18960602]
 ...
 [0.0530283  0.1274861  0.08400815 ... 1.         0.19210501 0.17345946]
 [0.03449277 0.21825991 0.23456407 ... 0.19210501 1.         0.29102505]
 [0.04824983 0.18299363 0.18960602 ... 0.17345946 0.29102505 1.        ]]


In [15]:
item_similarity_df = pd.DataFrame(item_similarity,index=ratings_2.columns,
                                 columns=ratings_2.columns)
item_similarity_df

,ADULT INCONTINENCE,AIR CARE,ANALGESICS,ANTACIDS,APPAREL,APPLES,AUDIO/VIDEO PRODUCTS,AUTOMOTIVE PRODUCTS,BABY FOODS,BABY HBC,...,VEAL,VEGETABLES - ALL OTHERS,VEGETABLES - SHELF STABLE,VEGETABLES SALAD,VITAMINS,WAREHOUSE SNACKS,WATCHES/CALCULATORS/LOBBY,WATER,WATER - CARBONATED/FLVRD DRINK,YOGURT
ADULT INCONTINENCE,1.000000,0.094544,0.067134,0.128257,0.049764,0.028143,0.026698,0.049558,-0.009668,0.018492,...,0.029217,0.063627,0.018532,0.002025,0.121154,0.035711,0.002011,0.053028,0.034493,0.048250
AIR CARE,0.094544,1.000000,0.281391,0.174372,0.132129,0.197019,0.123168,0.158366,0.085843,0.202300,...,0.028371,0.151035,0.190311,0.110703,0.195509,0.180629,0.045580,0.127486,0.218260,0.182994
ANALGESICS,0.067134,0.281391,1.000000,0.275503,0.110007,0.180574,0.122779,0.144244,0.103404,0.237561,...,0.016564,0.114695,0.168869,0.123666,0.261404,0.196245,0.013566,0.084008,0.234564,0.189606
ANTACIDS,0.128257,0.174372,0.275503,1.000000,0.076646,0.149057,0.069965,0.112653,0.020500,0.123847,...,0.066723,0.115907,0.097746,0.058124,0.192964,0.119709,-0.015141,0.057008,0.097497,0.096457
APPAREL,0.049764,0.132129,0.110007,0.076646,1.000000,0.066670,0.137903,0.093002,0.020344,0.078626,...,0.003038,0.077012,0.062318,0.043539,0.111750,0.101866,0.026086,0.008291,0.078198,0.096423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WAREHOUSE SNACKS,0.035711,0.180629,0.196245,0.119709,0.101866,0.198466,0.097149,0.110608,0.138724,0.167526,...,0.023383,0.130251,0.227389,0.167087,0.079804,1.000000,0.006222,0.091472,0.235883,0.199269
WATCHES/CALCULATORS/LOBBY,0.002011,0.045580,0.013566,-0.015141,0.026086,-0.013855,0.066424,0.018768,-0.026939,-0.014385,...,0.005501,-0.012038,0.044199,0.012363,-0.006447,0.006222,1.000000,0.017725,-0.034603,-0.031892
WATER,0.053028,0.127486,0.084008,0.057008,0.008291,0.133209,0.056311,0.060428,0.021537,0.079728,...,0.081522,0.098269,0.075101,0.053894,0.069900,0.091472,0.017725,1.000000,0.192105,0.173459
WATER - CARBONATED/FLVRD DRINK,0.034493,0.218260,0.234564,0.097497,0.078198,0.290774,0.103656,0.123747,0.127458,0.195158,...,0.044497,0.178623,0.197843,0.192612,0.121439,0.235883,-0.034603,0.192105,1.000000,0.291025


In [16]:
def get_similar_products(COMMODITY_DESC,user_rating):
    similar_score = item_similarity_df[COMMODITY_DESC]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    
    return similar_score
# print(get_similar_products('P1000002',5))

In [17]:
print(get_similar_products('APPLES',5))

APPLES                          2.500000
GRAPES                          1.157405
CITRUS                          1.065685
CARROTS                         1.046876
TROPICAL FRUIT                  1.037004
                                  ...   
WATCHES/CALCULATORS/LOBBY      -0.034637
SMOKING CESSATIONS             -0.043265
FROZEN PACKAGE MEAT            -0.076259
CIGARETTES                     -0.119220
PREPAID WIRELESS&ACCESSORIES   -0.141065
Name: APPLES, Length: 304, dtype: float64


In [18]:
# 測試資料
# 對三個產品評價，推薦產品給客戶

In [19]:
test = [("BABY FOODS",1),('AUTOMOTIVE PRODUCTS',4),('ANTACIDS',2)]

similar_products = pd.DataFrame()
for product,rating in test:
    similar_products = similar_products.append(get_similar_products(product,rating))
similar_products.head()

,ADULT INCONTINENCE,AIR CARE,ANALGESICS,ANTACIDS,APPAREL,APPLES,AUDIO/VIDEO PRODUCTS,AUTOMOTIVE PRODUCTS,BABY FOODS,BABY HBC,...,VEAL,VEGETABLES - ALL OTHERS,VEGETABLES - SHELF STABLE,VEGETABLES SALAD,VITAMINS,WAREHOUSE SNACKS,WATCHES/CALCULATORS/LOBBY,WATER,WATER - CARBONATED/FLVRD DRINK,YOGURT
BABY FOODS,0.014502,-0.128764,-0.155106,-0.030751,-0.030516,-0.110509,-0.086422,-0.071345,-1.500000,-0.563066,...,-0.014820,-0.116305,-0.139252,-0.111523,-0.007685,-0.208086,0.040408,-0.032306,-0.191187,-0.123450
AUTOMOTIVE PRODUCTS,0.074337,0.237548,0.216367,0.168979,0.139503,0.141737,0.163372,1.500000,0.071345,0.239668,...,0.052110,0.136416,0.109767,0.101028,0.239162,0.165911,0.028151,0.090642,0.185620,0.135650
ANTACIDS,-0.064128,-0.087186,-0.137751,-0.500000,-0.038323,-0.074528,-0.034982,-0.056326,-0.010250,-0.061923,...,-0.033362,-0.057953,-0.048873,-0.029062,-0.096482,-0.059855,0.007571,-0.028504,-0.048749,-0.048229


In [20]:
similar_products.sum().sort_values(ascending=False)

AUTOMOTIVE PRODUCTS             1.372329
HARDWARE SUPPLIES               0.270415
CEREAL/BREAKFAST                0.201003
ELECTRICAL SUPPPLIES            0.170526
DELI SPECIALTIES (RETAIL PK)    0.168642
                                  ...   
BABY HBC                       -0.385321
INFANT CARE PRODUCTS           -0.600026
DIAPERS & DISPOSABLES          -0.662652
INFANT FORMULA                 -0.772512
BABY FOODS                     -1.438905
Length: 304, dtype: float64

### 使用pearson 相關係數計算相似度

In [21]:
user_ratings = ratings.pivot_table(values='rating_label', index=['household_key'], columns=['COMMODITY_DESC'])

# ratings_2 = DfMatrix.dropna(thresh=10,axis=1).fillna(0)# 產品評價次數不滿10次的移除
ratings_2 = DfMatrix.reset_index().set_index('household_key').rename_axis(None, axis=1)
ratings_2 = ratings_2.fillna(0) 
ratings_2

,ADULT INCONTINENCE,AIR CARE,ANALGESICS,ANTACIDS,APPAREL,APPLES,AUDIO/VIDEO PRODUCTS,AUTOMOTIVE PRODUCTS,BABY FOODS,BABY HBC,...,VEAL,VEGETABLES - ALL OTHERS,VEGETABLES - SHELF STABLE,VEGETABLES SALAD,VITAMINS,WAREHOUSE SNACKS,WATCHES/CALCULATORS/LOBBY,WATER,WATER - CARBONATED/FLVRD DRINK,YOGURT
household_key,,,,,,,,,,,,,,,,,,,,,
1,0.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0
2,5.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,...,0.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0
3,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,...,0.0,0.0,5.0,5.0,0.0,5.0,0.0,0.0,5.0,5.0
4,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
5,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0,...,0.0,5.0,5.0,5.0,0.0,5.0,0.0,0.0,5.0,5.0
2497,0.0,5.0,5.0,1.0,0.0,5.0,0.0,4.0,5.0,1.0,...,0.0,5.0,5.0,5.0,0.0,1.0,0.0,0.0,5.0,5.0
2498,0.0,5.0,5.0,5.0,2.0,4.0,0.0,0.0,0.0,0.0,...,0.0,5.0,5.0,0.0,0.0,2.0,0.0,0.0,5.0,5.0


In [22]:
item_similarity_df = user_ratings.corr(method='pearson')
item_similarity_df.head()

COMMODITY_DESC,ADULT INCONTINENCE,AIR CARE,ANALGESICS,ANTACIDS,APPAREL,APPLES,AUDIO/VIDEO PRODUCTS,AUTOMOTIVE PRODUCTS,BABY FOODS,BABY HBC,...,VEAL,VEGETABLES - ALL OTHERS,VEGETABLES - SHELF STABLE,VEGETABLES SALAD,VITAMINS,WAREHOUSE SNACKS,WATCHES/CALCULATORS/LOBBY,WATER,WATER - CARBONATED/FLVRD DRINK,YOGURT
COMMODITY_DESC,,,,,,,,,,,,,,,,,,,,,
ADULT INCONTINENCE,1.000000,0.279288,0.062021,0.581501,0.695614,0.068809,0.548511,0.565537,0.002439,0.326195,...,0.700649,0.156760,0.125114,0.191628,0.422257,0.203905,-0.188982,0.308106,0.035539,0.115715
AIR CARE,0.279288,1.000000,0.198770,0.338316,0.284649,0.172011,0.319436,0.309513,0.155628,0.181040,...,0.544278,0.194976,0.113793,0.108260,0.202953,0.113818,0.328836,0.148133,0.185608,0.104805
ANALGESICS,0.062021,0.198770,1.000000,0.335345,0.282734,0.232235,0.323163,0.184386,0.135301,0.252739,...,0.378738,0.089689,0.077263,0.179072,0.265881,0.115265,0.170283,0.134633,0.110209,0.185872
ANTACIDS,0.581501,0.338316,0.335345,1.000000,0.282520,0.181875,0.335588,0.404451,0.110683,0.218565,...,0.390706,0.192559,0.026874,0.189571,0.352530,0.044574,0.221637,0.264074,0.065165,0.177209
APPAREL,0.695614,0.284649,0.282734,0.282520,1.000000,0.203752,0.435283,0.376240,0.276413,0.271164,...,0.345597,0.075755,0.103134,0.276249,0.327357,0.188394,0.342527,0.053199,0.120881,0.072687


In [23]:
def get_similar_products(COMMODITY_DESC,user_rating):
    similar_score = item_similarity_df[COMMODITY_DESC]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    
    return similar_score

In [24]:
test = [("BABY FOODS",1),('AUTOMOTIVE PRODUCTS',4),('ANTACIDS',2)]

similar_products = pd.DataFrame()
for product,rating in test:
    similar_products = similar_products.append(get_similar_products(product,rating))
similar_products.head()
similar_products.sum().sort_values(ascending=False)

SMOKING CESSATIONS             1.559247
BOUQUET (NON ROSE)             1.290343
NATURAL VITAMINS               1.271576
PHARMACY                       1.135931
AUTOMOTIVE PRODUCTS            1.078019
                                 ...   
BOTTLE DEPOSITS               -0.999437
RW FRESH PROCESSED MEAT       -1.039966
BABY FOODS                    -1.335586
LONG DISTANCE CALLING CARDS   -1.341555
TOYS                          -2.757942
Length: 304, dtype: float64